# T5 on TPU 💥🚀

In [28]:
VERSION = "20211010"  #@param ["1.5" , "20211010", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  37072      0 --:--:-- --:--:-- --:--:-- 37072
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20211010 ...
Found existing installation: torch 1.10.0+cu111
     |████████████████████████████████| 57 kB 2.8 MB/s 
Uninstalling torch-1.10.0+cu111:
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.293 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 1.8.0 which is incompati

In [29]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.2 MB/s 


Install transformers and the nlp package. Restart colab after this

In [30]:
!git clone https://github.com/huggingface/transformers.git
!pip install ./transformers
!pip install -U nlp

Cloning into 'transformers'...
remote: Enumerating objects: 95470, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 95470 (delta 4), reused 12 (delta 4), pack-reused 95457
Receiving objects: 100% (95470/95470), 80.67 MiB | 26.21 MiB/s, done.
Resolving deltas: 100% (69290/69290), done.
Processing ./transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.8 MB 5.2 MB/s 
     |████████████████████████████████| 596 k

     |████████████████████████████████| 1.7 MB 5.2 MB/s 


## Load and process data

In [1]:
import os 
os.environ['LD_LIBRARY_PATH']='/usr/local/lib'

!echo $LD_LIBRARY_PATH
!sudo ln -s /usr/local/lib/libmkl_intel_lp64.so /usr/local/lib/libmkl_intel_lp64.so.1
!sudo ln -s /usr/local/lib/libmkl_intel_thread.so /usr/local/lib/libmkl_intel_thread.so.1
!sudo ln -s /usr/local/lib/libmkl_core.so /usr/local/lib/libmkl_core.so.1

!ldconfig
!ldd /usr/local/lib/python3.7/dist-packages/torch/lib/libtorch.so

/usr/local/lib
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

	linux-vdso.so.1 (0x00007ffdcfd30000)
	/usr/lib/x86_64-linux-gnu/libtcmalloc.so.4 (0x00007f845ad22000)
	libtorch_cpu.so => /usr/local/lib/python3.7/dist-packages/torch/lib/libtorch_cpu.so (0x00007f84503dc000)
	libc10.so => /usr/local/lib/python3.7/dist-packages/torch/lib/libc10.so (0x00007f845017e000)
	libmkl_intel_lp64.so.1 => /usr/local/lib/libmkl_intel_lp64.so.1 (0x00007f844f64d000)
	libmkl_intel_thread.so.1 => /usr/local/lib/libmkl_intel_thread.so.1 (0x00007f844d16d000)
	libmkl_core.so.1 => /usr/local/lib/libmkl_core.so.1 (0x00007f8449034000)
	libomp.so.5 => /usr/lib/x86_64-linux-gnu/libomp.so.5 (0x00007f8448d7f000)
	libpthread.so.0 => /lib/x86_64-linux-gnu/libpthread.so.0 (0x00007f8448b60000)
	libm.so.6 => /lib/x86_64-linux-gnu/libm.so.6 (0x00007f84487c2000)
	libdl.so.2 => /lib/x86_64-linux-gnu/libdl.so.2 (0x00007f84485be000)
	libstdc++.so.6 => /usr/lib/x8

In [2]:
import torch
import nlp
from transformers import T5Tokenizer

In [3]:
# process the examples in input and target text format and the eos token at the end 
def add_eos_to_examples(example):
    example['input_text'] = 'question: %s  context: %s </s>' % (example['question_translated'], example['context_translated'])
    example['target_text'] = '%s </s>' % example['text_translated']
    return example

# tokenize the examples
def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['input_text'], pad_to_max_length=True, max_length=1024)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target_text'], pad_to_max_length=True, max_length=32)

    encodings = {
        'input_ids': input_encodings['input_ids'], 
        'attention_mask': input_encodings['attention_mask'],
        'target_ids': target_encodings['input_ids'],
        'target_attention_mask': target_encodings['attention_mask']
    }
    

    return encodings

# Evaluation on SLQuAD dataste

In [5]:
## SQuAD evaluation script. Modifed slightly for this Sinhala QA task

from __future__ import print_function
from collections import Counter
import string
import re
import argparse
import json
import sys


def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    # prediction_tokens = normalize_answer(prediction).split()
    # ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction) & Counter(ground_truth)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction)
    recall = 1.0 * num_same / len(ground_truth)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (prediction == ground_truth)


def metric_max_over_ground_truths(metric_fn, prediction, ground_truth):
    scores_for_ground_truths = []
    score = metric_fn(prediction, ground_truth)
    scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)


def evaluate(gold_answers, predictions):
    f1 = exact_match = total = 0

    for ground_truths, prediction in zip(gold_answers, predictions):
      total += 1
      exact_match += metric_max_over_ground_truths(
                    exact_match_score, prediction, ground_truths)
      f1 += metric_max_over_ground_truths(
          f1_score, prediction, ground_truths)
    
    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    return {'exact_match': exact_match, 'f1': f1}

In [6]:
import torch
import torch_xla
import torch_xla.core.xla_model as xm

import nlp
from transformers import T5ForConditionalGeneration, T5Tokenizer

from tqdm.auto import tqdm

In [8]:
model = T5ForConditionalGeneration.from_pretrained('sankhajay/mt5-base-sinaha-qa').to('cpu') # because its loaded on xla by default
tokenizer = T5Tokenizer.from_pretrained('sankhajay/mt5-base-sinaha-qa')
# tokenizer = T5Tokenizer.from_pretrained('models/tpu')

Downloading:   0%|          | 0.00/680 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.07G [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torch/_utils.py:134: UserWarning: Failed to initialize NumPy: module compiled against API version 0xe but this version of numpy is 0xd (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:68.)
  t = torch.tensor([], dtype=storage.dtype, device=storage._untyped().device)


Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/431 [00:00<?, ?B/s]

In [9]:
import json
columns = ['question_translated','text_translated','context_translated','answer_start','answer_end']

In [10]:
!pip install datasets

In [31]:
question = 'ශ්‍රී ලංකාව පිහිටා ඇත්තේ කොහෙද ?'

context = 'ශ්‍රී ලංකාව යනු ඉන්දියානු සාගරයේ පිහිටි මනරම් දුපතකි.'

answer = 'ඉන්දියානු සාගරයේ'

In [32]:
import json
columns = ['question_translated','text_translated','context_translated','answer_start','answer_end']

In [33]:
import csv
import pandas as pd

header = ['answer_end', 'answer_start', 'context_translated', 'question_translated','text_translated']
data = [0,0,context,question,answer]

with open('predict.csv', 'w', encoding='UTF8') as f:
  writer = csv.writer(f)
  writer.writerow(header)
  writer.writerow(data)

predict = pd.read_csv('predict.csv')

In [34]:
from datasets import Dataset

predict = Dataset.from_pandas(predict.loc[0:,columns])

In [35]:
predict[0]

{'answer_end': 0,
 'answer_start': 0,
 'context_translated': 'ශ්\u200dරී ලංකාව යනු ඉන්දියානු සාගරයේ පිහිටි මනරම් දුපතකි.',
 'question_translated': 'ශ්\u200dරී ලංකාව පිහිටා ඇත්තේ කොහෙද ?',
 'text_translated': 'ඉන්දියානු සාගරයේ'}

In [36]:
# process the examples in input and target text format and the eos token at the end 
def add_eos_to_examples(example):
    example['input_text'] = 'question: %s  context: %s </s>' % (example['question_translated'], example['context_translated'])
    example['target_text'] = '%s </s>' % example['text_translated']
    return example

# tokenize the examples
def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['input_text'], pad_to_max_length=True, max_length=1024)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target_text'], pad_to_max_length=True, max_length=32)

    encodings = {
        'input_ids': input_encodings['input_ids'], 
        'attention_mask': input_encodings['attention_mask'],
        'target_ids': target_encodings['input_ids'],
        'target_attention_mask': target_encodings['attention_mask']
    }
    

    return encodings

In [37]:
predict = predict.map(add_eos_to_examples, load_from_cache_file=False)
predict = predict.map(convert_to_features, batched=True, load_from_cache_file=False)


# set the tensor type and the columns which the dataset should return
columns = ['input_ids', 'target_ids', 'attention_mask', 'target_attention_mask']
predict.set_format(type='torch', columns=columns)

  0%|          | 0/1 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2226: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:195: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


In [38]:
predict

Dataset({
    features: ['answer_end', 'answer_start', 'attention_mask', 'context_translated', 'input_ids', 'input_text', 'question_translated', 'target_attention_mask', 'target_ids', 'target_text', 'text_translated'],
    num_rows: 1
})

In [39]:
len(predict)

1

In [40]:
# cach the dataset, so we can load it directly for training

torch.save(predict, 'predict_data.pt')

In [41]:
predict[0]['input_ids']

tensor([ 7680,   267, 25454,  ...,     0,     0,     0])

In [42]:
predict_dataset = torch.load('predict_data.pt')
dataloader = torch.utils.data.DataLoader(predict_dataset, batch_size=1)

In [43]:
answers = []
for batch in tqdm(dataloader):
  outs = model.generate(input_ids=batch['input_ids'], 
                        attention_mask=batch['attention_mask'],
                        max_length=32,
                        early_stopping=True)
  outs = [tokenizer.decode(ids) for ids in outs]
  answers.extend(outs)

  0%|          | 0/1 [00:00<?, ?it/s]

In [44]:
predictions = []
references = []

for ref, pred in zip(predict_dataset['text_translated'], answers):
  pred = pred.replace("<pad>", "").replace("</s>", "").rstrip().lstrip()
  predictions.append(pred)
  references.append(ref)

In [45]:
print('ground truth: ',references[0])

print('predicted: ',predictions[0])

ground truth:  ඉන්දියානු සාගරයේ
predicted:  ඉන්දියානු සාගරය


In [46]:
evaluate(references, predictions)

{'exact_match': 0.0, 'f1': 96.7741935483871}